In [1]:
import numpy as np
import scipy.sparse
from scipy.sparse.linalg import lobpcg, eigsh
from scipy.sparse import csr_matrix

In [8]:
### Build adjacency matrix from counts
print("Building the word2id index..")
vocab_path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1.words.vocab"
word2id = {}
with open(vocab_path, 'r') as f:
    for idx, line in enumerate(f):
        split_line = line.strip().split(",")
        word = ",".join(split_line[:len(split_line)-1])
        word2id[word] = idx

print("Reading the count data...")
N = len(word2id)
counts_path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1"
data = []
rows = []
cols = []

with open(counts_path, 'r') as f:
    for line in f:
        count, word_a, word_b = line.strip().split()
        word_a_id = word2id[word_a]
        word_b_id = word2id[word_b]
        
        data.append(float(count))
        rows.append(word_a_id)
        cols.append(word_b_id)

print("Building the adjacency matrix...")
adjacency_matrix = csr_matrix((data, (rows, cols)), shape=[N,N])
print("Done!")

Building the word2id index..
Reading the count data...
Building the adjacency matrix...
Done!


In [2]:
adj_path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1.adj"
#scipy.sparse.save_npz(adj_path, adjacency_matrix)
adjacency_matrix = scipy.sparse.load_npz(adj_path + ".npz")

In [ ]:
### Find spectral embeddings
d=300
#max_iter=1000
#verbosity=1
#seed=0

degrees = adjacency_matrix.sum(axis=1).flatten()
n = adjacency_matrix.shape[0]
D = scipy.sparse.spdiags(degrees, [0], n, n, format='csr')
L = D - adjacency_matrix
degrees_sqrt = 1.0 / scipy.sqrt(degrees)
DH = scipy.sparse.spdiags(degrees_sqrt, [0], n, n, format='csr')
L_norm = DH.dot(L.dot(DH))

#rng = np.random.RandomState(seed)
#init = rng.rand(n, d + 1)
#init[:,0] = 1.0/adjacency_matrix.sum(axis=1).flatten()
#vals, vecs = lobpcg(A=L_norm, X=init, largest=False, maxiter=max_iter, verbosityLevel=verbosity)
vals, vecs = eigsh(L_norm, k=d+1, which="SM")

# eigen_scaling = 1.0 / (vals[1:])
# rescaled_eigenvectors = np.sqrt(eigen_scaling) * vecs[:, 1:] / np.sqrt(np.asarray(degrees).T)
# rescaled_eigenvectors = np.ascontiguousarray(rescaled_eigenvectors)

In [ ]:
# save_path = "/Users/i.lobov/hyperwords/data/wiki/spectral_embeddings_d=300.words"
# np.save(save_path, rescaled_eigenvectors)

# save_path = "/Users/i.lobov/hyperwords/data/wiki/eigenvectors_d=500.words"
# np.save(save_path, vecs)

# save_path = "/Users/i.lobov/hyperwords/data/wiki/eigenvalues_d=500.words"
# np.save(save_path, vals)

In [3]:
# save_path = "/Users/i.lobov/hyperwords/data/wiki/eigenvectors_d=300.words.npy"
# vecs = np.load(save_path)

# save_path = "/Users/i.lobov/hyperwords/data/wiki/eigenvalues_d=300.words.npy"
# vals = np.load(save_path)

# degrees = adjacency_matrix.sum(axis=1).flatten()

In [13]:
eigen_scaling = 1.0 / (vals[1:])
rescaled_eigenvectors = np.sqrt(eigen_scaling) * vecs[:, 1:] / np.sqrt(np.asarray(degrees).T)
rescaled_eigenvectors = np.ascontiguousarray(rescaled_eigenvectors)

In [15]:
path = "/Users/i.lobov/hyperwords/testsets/ws/ws353.txt"
test = []
with open(path) as f:
    for line in f:
        x, y, sim = line.strip().lower().split()
        test.append(((x, y), float(sim)))

In [5]:
from representations.matrix_serializer import load_vocabulary

path = "/Users/i.lobov/hyperwords/data/wiki/wikipedia.corpus.nodups_counts_win=1.words.vocab"
wi, iw = load_vocabulary(path)

In [47]:
from scipy.stats.stats import spearmanr

m = np.sqrt(eigenvals[1:]) * vecs[:,1:].copy()
m = m / np.linalg.norm(m, axis=1, keepdims=True)
dim = 300
C = degrees.sum() / 10.0

def represent(w):
    if w in wi:
        return m[wi[w], :]
    else:
        return np.zeros(dim)

# def similarity(w1, w2):
#     rep_w1 = represent(w1)
#     rep_w2 = represent(w2)
#     inner_product = np.log(max(C * rep_w1.dot(rep_w2), 1.0))
#     w1_l2 = np.log(C * rep_w1.dot(rep_w1))
#     w2_l2 = np.log(C * rep_w2.dot(rep_w2))
#     #print(inner_product, rep_w1.dot(rep_w2))
    
#     return inner_product/np.sqrt(w1_l2*w2_l2)


def similarity(w1, w2):
    return represent(w1).dot(represent(w2))

results = []
for (x, y), sim in test:
    results.append((similarity(x, y), sim))
actual, expected = zip(*results)
spearmanr(actual, expected)[0]

0.21888865179835149

In [33]:
window = 5

eigenvals = np.zeros_like(vals)
for r in range(1, window+1):
    eigenvals += vals**r
eigenvals /= window

In [4]:
actuals

NameError: name 'actuals' is not defined

In [47]:
np.log(1e-6)

-13.815510557964274